In [1]:
# @zhangjoy
# @构建主题热度因子：201802【论文】
import os
import csv
import tushare as ts
import pandas as pd
import numpy as np
import jieba
import sys
import time
import re
from sklearn.feature_extraction.text import CountVectorizer # 词频统计
from sklearn.decomposition import LatentDirichletAllocation # LDA
from sklearn.cluster import KMeans # k均值聚类

In [47]:
# 评论文本：comment
filepath = "/Users/zhangjoy/Documents/vscode/py_project/lda/data/cjpl/20160401_20160501.csv"   # 评论内容的存储路径
with open(filepath, 'r') as f:
    reader = csv.reader(f)
    comment = [row[7] for row in reader]
comment.pop(0) # '评论内容'
print(len(comment)) # 2009

2009


In [48]:
comment[0]

'来源：中国基金报 编辑：东方财富网股市29日收盘，“红四月”再次爽约：上半月大盘震荡上行，下半月则判若两人一路阴跌不止，最终上证综指和创业板指在四月均以小幅下跌收场。债市风险暴露，一时间风声鹤唳，机构惊慌失色。尽管今年前4月股市和债市表现动荡，公募基金表现分化很大，但前四月里还是有一些基金取得了不错的正收益，基金君跟大家一起来看下吧。1、主动股票型基金：平均跌16.44% 四只次新基金逆势获正收益今年前4月，148只主动股票型基金单位净值平均下跌了16.44%，基金分化十分明显，其中有四只去年四季度成立的圆信永丰优加生活、嘉实环保低碳、工银文体产业和泓德战略转型等次新基金凭借较为稳健的建仓节奏，在今年前四月取得了正回报，其余基金则全部告亏，特别是有三只基金亏损超过30%，今年减亏压力甚大！2、混合型基金： 273只跌幅超两成最多跌35.57%混合型基金现在是数量最多的基金类别，各类混合型基金总数已超过1000只。前4月，混合基金分化同样十分明显。10只表现最好的混合基金前4月收益率均在5%以上，这在今年这样的动荡市场下着实难得，前海开源金银珠宝赚了12.55%居首，主要是受益金价大涨刺激下相关个股不俗的表现。此外还有不少数量的混合基金取得了正收益。但是，跌幅较大的混合基金占比也较高，其中有273只前四月跌幅超过20%，特别是有16只混合基金跌幅超过30%，远超市场。3、打新基金：部分基金靠赎回费大赚今年前4月，有相当数量的绝对收益策略基金(以打新基金为主)在遭遇大比例净赎回之后依靠赎回费计入资产净值飙升，成为今年以来最赚钱的基金群体，在别人家的基金跌的一塌糊涂的时候，这些基金在大口吃肉。原来，除了股票，我们还可以赚赎回费。潜伏那些机构占比高的打新基金成为一些投资者热捧的盈利模式。基金君注意到，一些主流财经媒体竟然将这些基金当做今年的绩优基金加以报道，分析其一季报的持仓、重仓股、观点等等，基金君也只能呵呵了.4、保本基金：好发不好做多只基金亏超3%保本基金毫无疑问是股灾以来基金产品中的爆款，投资者对本金保护的强烈需求使得这类产品已经热卖了大半年。但股市和债市的双重大震荡使得保本基金陷入好发不好做的困境。今年前4月，92只保本基金平均小幅亏损1.14%，一些基金实现了正收益，收益最高的新华阿里一号赚了1.67%，其他取得正收益的保本基金净值涨幅均在0.8%以下。大部分

In [49]:
# 得到所有股票的代码和中文名字
stock_name = ts.get_stock_basics()['name'] # 股票名称
stock_code = stock_name.index.tolist() # 股票代码
# 获得所有的概念列表、行业列表、地域列表、公司名称等
concept = ts.get_concept_classified()['c_name'].drop_duplicates().values.tolist() # 概念列表 163
industry = ts.get_industry_classified('sw')['c_name'].drop_duplicates().values.tolist() # 申万行业列表 129 > 110
place = ts.get_area_classified()['area'].drop_duplicates().values.tolist() # 获取地域列表 32 省份

broker = ts.broker_tops()["broker"] # 获取营业部信息：1062
pattern = re.compile(u'.*有限.*?公司') # 最小匹配[*?]到'有限 公司'
company = [" ".join(pattern.findall(s)) for s in broker] # list
company = pd.Series(company).drop_duplicates().values.tolist() # 证券公司：94

[Getting data:]################################

In [50]:
# 启用自定义词典
jieba.load_userdict("/Users/zhangjoy/Documents/vscode/py_project/lda/data/userdict.txt")
# 停用词列表
stop =[line.strip() for line in open('/Users/zhangjoy/Documents/vscode/py_project/lda/data/stopwords.txt').readlines()]

In [51]:
pattern = re.compile(u'[\u4e00-\u9fa5]{2,20}') # 中文

In [52]:
texts = [] # 清洗后文本
t1 = time.time()
for sentence in comment:
    chinese = pattern.findall(sentence) # 匹配中文：先分句
    seg = [jieba.cut(c) for c in chinese] # 双重列表：句上分词
    c = []
    for ss in seg:
        for s in ss:
            if s not in stop: # 过滤停用词
                c.append(s)
    texts.append(" ".join(c))
t2 = time.time()
print("preprocessing time using %.5f s" %(t2-t1)) # 2000 words < 1m【6000 2m】
print(texts[0])

preprocessing time using 46.06072 s
股市 收盘 红四月 爽约 大盘 震荡 上行 判若两人 一路 阴跌 上证综指 创业板指 均 小幅 下跌 收场 债市 风险 暴露 一时间 风声鹤唳 机构 惊慌失色 前 股市 债市 表现 动荡 公募基金 表现 分化 很大 但前 里 基金 取得 不错 正收 基金君 一起 主动 股票型基金 平均 跌 次新基金 逆势 获 正收益 前 主动 股票型基金 单位 净值 平均 下跌 基金 分化 十分 明显 成立 圆信 永丰 优加生 嘉实 环保 低碳 工银 文体 产业 和泓德 战略转型 次新基金 稳健 建仓 节奏 前 取得 正回报 基金 告亏 特别 基金 亏损 减亏 压力 甚大 混合型基金 跌幅 超 两成 最多 跌 混合型基金 数量 最多 基金 类别 各类 混合型基金 总数 混合基金 分化 同样 十分 明显 表现 最好 混合基金 前 收益率 均 动荡市场 着实 前海 开源 金银珠宝 赚 居首 主要 受益 金价 大涨 刺激 相关 个股 不俗 表 数量 混合基金 取得 正收益 跌幅 较大 混合基金 占 高 只前 跌幅 特别 混合基金 跌幅 远超 打新基金 部分 基金 赎回费 赚 前 相当 数量 绝对收益 策略 基金 打新基金 为主 遭遇 比例 净 赎回 依靠 赎回费 计入 资产 净值 飙升 成为 今年以来 赚钱 基金 群体 家 基金 跌 一塌糊涂 基金 口吃 肉 原来 股票 赚 赎回费 潜伏 机构 占 比高 打新基金 成为 投资 热 捧 盈利模式 基金君 注意 主流 财经 媒体 基金 当做 绩优 基金 报道 分析 季报 持仓 重仓股 观点 基金君 只能 保本基金 好发 不好 做 多只 基金 亏超 保本基金 毫无疑问 股灾 基金 产品 中 爆 投资者 本金 保护 强烈 需求 类产品 热卖 大半年 股市 债市 双 重大 震荡 保本基金 陷入 好发 不好 做 困境 前 保本基金 平均 小幅 亏损 基金 实现 正收益 收益 最高 新华 阿里 一号 赚 取得 正收益 保本基金 净值 涨幅 均 保本基金 前 亏损 跌幅 跌幅 最大 大跌眼镜 纯债基金 遭遇 黑色 平均 收益 由正 转负 纯债基金 表现 债市 遭遇 黑色 纯债基金 今年以来 收益 大幅 下滑 统计 显示 余 纯债基金 前 平均 收益 跌为 零 有近 取得 正收益 收益

In [55]:
# 建模
corpus = texts
# 构建 Document-word matrix
count_vectorizer = CountVectorizer(stop_words=stop, min_df = 0.005) # 特征选择：必须在文档总数的 0.5% 中出现过
count_matrix = count_vectorizer.fit_transform(corpus)
wordnames_list = count_vectorizer.get_feature_names() # word_list
count_matrix.shape

(2009, 6586)

In [56]:
# 训练模型
k = 20 # 选择主题数
lda = LatentDirichletAllocation(n_components=k, max_iter=50,learning_method='batch',random_state=1234)
%time lda.fit(count_matrix) # 2000 1m【6000 3m】

CPU times: user 59.9 s, sys: 453 ms, total: 1min
Wall time: 1min 1s


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=20, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=1234, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [16]:
# Doc（Top）Topic结果
def print_top_topics(model, feature_names, n_top_topics):
    print("# 打印每个文档下权重较高的主题")
    doctopic_matrix = model.fit_transform(count_matrix)
    for doc_idx, doc in enumerate(doctopic_matrix):
        print("Doc # %d:" % doc_idx)
        print(''.join([str(feature_names[i]) + ' ' + str(round(doc[i], 2)) + '| ' for i in doc.argsort()[:-n_top_topics - 1:-1]]))
# Topic（Top）Words结果
def print_top_words(model, feature_names, n_top_words):
    print("# 打印每个主题下权重较高的词")
    topicword_matrix = model.components_
    for topic_idx, topic in enumerate(topicword_matrix):
        print("Topic # %d: " % topic_idx)
        print(''.join([feature_names[i] + ' ' + str(round(topic[i], 2)) + '| ' for i in topic.argsort()[:-n_top_words - 1:-1]]))
        # print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
# 全部文档前几个主题，假设1个文档可以属于多个主题，首先一定属于1个主题，后面几个主题只要满足在1.5倍的差距内，即 0.4*1.5 < 0.6 0.3*1.5 0.5
def print_top_topics_number(model, feature_names, toptopicnames_list):
    doctopic_matrix = model.fit_transform(count_matrix)
    for doc_idx, doc in enumerate(doctopic_matrix):
        for i in doc.argsort()[:-k - 1:-1]:
            if i == doc.argsort()[:-k - 1:-1][0]:
                toptopicnames_list.append(feature_names[i])
                continue
            elif round(doc[i], 2)*1.5 >= round(doc[doc.argsort()[:-k - 1:-1][0]], 2):
                toptopicnames_list.append(feature_names[i])

In [57]:
topicnames_list = [i for i in range(k)] # topic_list
toptopicnames_list = []
print_top_topics_number(lda,topicnames_list,toptopicnames_list) # 全部文档前几个主题
# len(toptopicnames_list) # 

In [58]:
# 前 5个 热门主题
from collections import Counter
print(Counter(toptopicnames_list).most_common(20))
print(Counter(toptopicnames_list).most_common(5))

[(0, 289), (3, 270), (8, 236), (17, 199), (2, 185), (1, 169), (10, 140), (9, 134), (18, 132), (4, 130), (13, 130), (14, 122), (15, 120), (11, 99), (6, 98), (7, 78), (19, 77), (16, 60), (5, 45), (12, 44)]
[(0, 289), (3, 270), (8, 236), (17, 199), (2, 185)]


In [59]:
# 选择前 5 个主题：
topicnames = [Counter(toptopicnames_list).most_common(5)[i][0] for i in range(5)]
# 主题权重归一化
topicweight = [Counter(toptopicnames_list).most_common(5)[i][1] for i in range(5)]
topicweight = [i/sum(topicweight) for i in topicweight]

In [60]:
topicnames

[0, 3, 8, 17, 2]

In [61]:
topicweight

[0.24512298558100085,
 0.22900763358778625,
 0.2001696352841391,
 0.16878710771840544,
 0.15691263782866835]

In [62]:
# 3.热门主题关键词筛选，为每个主题选择前 N 个主题词
def print_hottopic_keywords(model, feature_names, topickeywords_name, topickeywords_prob, n_top_words):
    topicword_matrix = model.components_[topicnames]
    for topic_idx,topic in enumerate(topicword_matrix):
        topickeywords_n = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        topickeywords_name.append(topickeywords_n)
        topickeywords_p = [round(topic[i], 2) for i in topic.argsort()[:-n_top_words - 1:-1]]
        topickeywords_prob.append(topickeywords_p)

In [63]:
topickeywords_name, topickeywords_prob = [],[]
print_hottopic_keywords(lda, wordnames_list, topickeywords_name, topickeywords_prob , 100) # 选前100个关键词

In [77]:
for i in range(100):
    print(topickeywords_name[2][i],topickeywords_prob[2][i])
# print()

改革 1470.71
发展 1041.18
金融 969.45
企业 645.65
地方 638.13
试点 534.91
制度 436.83
建设 430.02
国家 427.6
国务院 415.28
经济 391.02
监管 386.74
创新 381.69
加强 368.65
会议 360.92
政府 354.32
完善 341.51
支持 337.26
推动 318.39
机制 314.13
中央 311.46
全国 306.82
重要 301.76
提出 290.14
社会 278.06
平台 273.89
促进 273.33
快鹿 272.97
部门 265.89
要求 262.54
意见 257.95
兑付 256.98
加快 253.97
开展 250.5
合作 242.1
习近平 238.32
体系 233.49
强调 232.74
重点 229.36
建立 229.03
实现 226.24
方面 219.06
互联网 215.59
项目 214.98
积极 211.06
进一步 210.73
方案 210.51
李克强 202.85
风险 202.3
服务 202.02
制定 201.77
立法 194.74
研究 193.18
我国 192.39
召开 190.19
提高 187.72
领域 186.12
地区 185.31
体制 182.48
指出 179.43
就业 174.85
规划 171.32
深化 169.62
保障 169.49
做好 168.31
相关 165.15
目标 160.9
基础 157.63
广东 157.6
结构性 155.89
进行 154.4
形成 154.31
协调 153.99
财政部 153.06
总理 152.46
行业 152.34
任务 151.8
方式 151.3
明确 150.57
补贴 148.19
稳定 147.97
范围 146.97
落实 146.83
有效 146.32
供给 145.82
农业 145.74
金融机构 143.35
坚持 143.13
规范 141.1
保护 139.41
指导 138.88
关系 137.35
央行 136.73
组织 134.67
中国 134.63
情况 134.21
决定 131.52
各地 131.4
经营 130.48
座谈会 129.28

In [38]:
# 股票池：沪深300
hs300 = ts.get_hs300s() 
hs300_code = hs300['code'].values.tolist()
hs300_name = hs300['name'].values.tolist()
concept_df = ts.get_concept_classified()
industry_df = ts.get_industry_classified('sw')

# 与股票池相关的概念
concept = concept_df.loc[concept_df['code'].isin(hs300_code)]['c_name'].drop_duplicates().values.tolist() # 146
# 与股票池相关的行业
industry = industry_df.loc[industry_df['code'].isin(hs300_code)]['c_name'].drop_duplicates().values.tolist() # 104

http://www.csindex.com.cn/uploads/file/autofile/closeweight/000300closeweight.xls


In [41]:
len(industry)

104

In [82]:
# 4 主题因子构建：概念类、行业类
# 计算每个主题：主题关键词的概念/行业 信息
def concept_industry_info(topickeywords_name, concept_stay_list, concept_prob_list, industry_stay_list, industry_prob_list, topickeywords_weight):
    for i in range(len(topickeywords_name)):
        concept_num, industry_num = 0, 0
        concept_stay, concept_prob = [], []
        industry_stay, industry_prob = [], []
        for j in range(100):
            name = topickeywords_name[i][j]
            prob = topickeywords_prob[i][j]
            for con in concept: # 与关键词相关概念 
                if name in con:
                    concept_stay.append(con)
                    concept_prob.append(prob)
#                     print(name,con,prob)
                    concept_num = concept_num + 1
            for ind in industry: # 与关键词相关行业 
                if name in ind:
                    industry_stay.append(ind)
                    industry_prob.append(prob)
                    print(name,ind,prob)
                    industry_num = industry_num + 1
            
        print()
        # 主题类别权重
        if concept_num > 0 and industry_num > 0: # and place_num > 0:
            topickeywords_weight = [0.5, 0.5]
        elif concept_num == 0:
            topickeywords_weight = [0, 1]
        else:
            topickeywords_weight = [1, 0]
        
        concept_stay_list.append(concept_stay)
        concept_prob_list.append(concept_prob)
        industry_stay_list.append(industry_stay)
        industry_prob_list.append(industry_prob)
        topickeywords_weight_list.append(topickeywords_weight)

In [83]:
concept_stay_list, concept_prob_list, industry_stay_list, industry_prob_list, topickeywords_weight_list = [],[],[],[],[]
concept_industry_info(topickeywords_name, concept_stay_list, concept_prob_list, industry_stay_list, industry_prob_list, topickeywords_weight)


银行 银行 261.23

金融 非银金融 969.45
金融 多元金融 969.45
建设 基础建设 430.02
建设 房屋建设 430.02
互联网 互联网传媒 215.59
服务 医疗服务 202.02
服务 采掘服务 202.02
服务 环保工程及服务 202.02
服务 休闲服务 202.02
服务 其他休闲服务 202.02
服务 汽车服务 202.02
基础 基础建设 157.63
农业 农业综合 145.74
经营 商业物业经营 130.48

工业 工业金属 315.07
房地产 房地产 194.97
房地产 房地产开发 194.97
贸易 商业贸易 176.28
贸易 贸易 176.28
食品 食品加工 153.53
食品 食品饮料 153.53

煤炭 煤炭开采 191.51
钢铁 钢铁 138.35
银行 银行 130.85
证券 证券 106.34
有色 有色金属 91.19



In [71]:
concept_stay_list[1]

['低碳经济',
 '循环经济',
 '金融改革',
 '水域改革',
 '油气改革',
 '国企改革',
 '民营银行',
 '资产注入',
 '外资背景',
 '高校背景']

In [ ]:
concept_df.loc[concept_df['name'] == '安通控股']['c_name'].tolist()
industry_df.loc[industry_df['code'] == '600179']['c_name'].tolist()

In [84]:
def list_sum(a, b): # 两列表对应元素相乘并求和
    s = sum([ a[i]*b[i] for i in range(len(a)) ])
    return s

In [86]:
# 5.匹配个股信息，匹配上即累计得分
# 获取单只股票的概念信息、行业信息 --> 从而得到股票主题得分
stock_topicscore_list = []
for stock_code in hs300_code:
    stock_concept = concept_df.loc[concept_df['code'] == stock_code]['c_name'].tolist()
    stock_industry = industry_df.loc[industry_df['code'] == stock_code]['c_name'].tolist()
    stock_topicscore = []
    for i in range(5): # 5个主题得分
        concept_stay = concept_stay_list[i]
        concept_prob = concept_prob_list[i]
        industry_stay = industry_stay_list[i]
        industry_prob = industry_prob_list[i]
        topickeywords_weight = topickeywords_weight_list[i]
#         print(topickeywords_weight)
        concept_score = sum([concept_prob[i] for i in range(len(concept_stay)) if concept_stay[i] in stock_concept])
#         print('concept_score: %d' % concept_score)
        industry_score = sum([industry_prob[i] for i in range(len(industry_stay)) if industry_stay[i] in stock_industry])
#         print('industry_score: %d' % industry_score)
        ciscore_list = [concept_score,industry_score]
#         print(ciscore_list)
        topicscore = list_sum(ciscore_list, topickeywords_weight)
#         print('topicscore: %d' % topicscore)
        stock_topicscore.append(topicscore)
#     print(stock_topicscore)
#     print()
    stock_topicscore = list_sum(stock_topicscore, topicweight)
    stock_topicscore_list.append(stock_topicscore)

In [88]:
stock_name

code
002931      N锋龙
000862     银星能源
000505     京粮控股
002323      雅百特
002076    雪 莱 特
002775     文科园林
002930     宏川智慧
603718     海利生物
603838     四通股份
300637     扬帆新材
002693     双成药业
002813     路畅科技
603722      阿科力
300634     彩讯股份
300653     正海生物
000613     大东海A
300648     星云股份
300504     天邑股份
002888     惠威科技
300692     中环环保
300584     海辰药业
603912      佳力图
603803     瑞斯康达
603322     超讯通信
603214      爱婴室
600929     湖南盐业
300727     润禾材料
300540     深冷股份
300487     蓝晓科技
300514      友讯达
          ...  
300436      广生堂
603266     天龙股份
000810     创维数字
600135     乐凯胶片
300557     理工光科
300631     久吾高科
300101     振芯科技
300680     隆盛科技
002208     合肥城建
002513     蓝丰生化
603081     大丰实业
300216     千山药机
600539     狮头股份
603032     德新交运
000703     恒逸石化
000533    万 家 乐
300116     坚瑞沃能
603733     仙鹤股份
603596      伯特利
603301     振德医疗
600806    *ST昆机
600432    *ST吉恩
300743     天地数码
300742     越博动力
000950    *ST建峰
000629    *ST钒钛
000511    *ST烯碳
603897     长城科技
603876     鼎胜新材
603773     沃格光电
Name: name, Length:

In [87]:
stock_topicscore_list
# 最大最小标准化：（原数据-最小值）/（最大值-最小值）
stock_topicscore_list = [ (i-min(stock_topicscore_list))/(max(stock_topicscore_list)-min(stock_topicscore_list)) for i in stock_topicscore_list]
stock_topicscore_list

[0.4777091346814292,
 0.0,
 0.19299950992093487,
 0.3927911995573563,
 0.3833762439895939,
 0.047506152179031,
 0.047506152179031,
 0.09380342991193083,
 0.4047014609082243,
 0.18722312904512076,
 0.0,
 0.3153544549468604,
 0.0968163738678847,
 0.41717029446159826,
 0.02230016265501589,
 0.317429186381756,
 0.06549442905991223,
 0.0991087972397987,
 0.3034183854407015,
 0.09139644054849204,
 0.0968163738678847,
 0.2938383143275362,
 0.030194298367134283,
 0.01265655330548488,
 0.4377656989774508,
 0.16362616544227462,
 0.19299950992093487,
 0.11447239282022273,
 0.018862189721995593,
 0.02230016265501589,
 0.02230016265501589,
 0.19388016907858338,
 0.03996125054093302,
 0.40372015877327216,
 0.26513601499268435,
 0.3867652196836638,
 0.13166597274143083,
 0.5243623376278126,
 0.04192581175761804,
 0.0968163738678847,
 0.010419398966245807,
 0.04928900426586075,
 0.11823198922099003,
 0.0175377450616494,
 0.3841520336063087,
 0.11765517180037631,
 0.05591453083490234,
 0.23041739545001

In [100]:
# 保存结果
topicfactor_df = pd.DataFrame(stock_topicscore_list)
topicfactor_df.columns = ['topicfactor']
hs300_name_df = pd.DataFrame(hs300_name)
hs300_name_df.columns = ['stockname']
file_df = hs300_name_df.join(topicfactor_df)
file_df.to_csv("/Users/zhangjoy/Documents/vscode/py_project/lda/stock_topicfactor_ex.csv",encoding='gbk')